In [9]:
import pandas as pd
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import ipfshttpclient
import os
import io

In [3]:
# Load dataset
data = pd.read_csv('data.csv')
data_bytes = data.to_csv(index=False).encode()  # Convert CSV data to bytes for encryption

# Generate AES encryption key
key = os.urandom(32)  # AES-256 requires a 32-byte key

# Encrypt dataset
iv = os.urandom(12)  # 12-byte nonce for GCM mode
cipher = Cipher(algorithms.AES(key), modes.GCM(iv), backend=default_backend())
encryptor = cipher.encryptor()
encrypted_data = encryptor.update(data_bytes) + encryptor.finalize()
tag = encryptor.tag  # Authentication tag for GCM

In [4]:
print(encrypted_data)

b'\xb3\xed\x0bD\xdc\xeb7\x1dP\r=\xf0\x1aw\xc0\xb3\x9d\xaa\xe9$\x03\xca:\xca;\x88E\xfd\x0c\x18c\x90\x15\x0b\x98?\x8e\xeb$\xa3\x9c\x18\x89\xa1\x85\x11\x14\xe7M\x95\xb7\x0f\xb4}\x17\xbcAS\x02S\x14 \xf0\xf7K\xc3^}K\xf5\x00\xbcr\x91(;0\x15`\xcc\n#m=\xdb\xf7-\xaf\xce\xd5\x97\xd3\x91\xb4,\xff\xb8\xc4\x8a\x1c\xcb\x93\x98\xb5DV\xfa\xf6\xe9R\x03\xa3\xad\xdf\x01M\xbf\x14\xa4Z\x1c$M\x84\xa1\x8f\xe4xG\x82"\xad\xe1\xa4\x92\x19U\x8a\xa3:\x8a\xf9\xd3\xf2\xb73\xd1\xcc\x8e\\\xc1\x8f\xd4\x8c1\xeb@C\x7f\x1d\x8c\xa5\x12\x02\x02$Y\xc2a[\xdc\x93\xc1\x9dj\xe4y\x82f\xa9\x80&z\xaa\x97\x15\x02\x04\xd8\x92\xb8wt\'\x84h\xb8Z[\xee\xc1z\x1b\x9f,:j\xc3!A\xab\x0f?d@\xd3r\xeb\xee]\xbc\xf3\xef\xb0\x14\x18\x01K=\x05;\xc3\xe0\xb0\xba\xb1\x0f\xa3\n\x8e\xe9\xe3o\xa2\x1c\xb5\xe8\x10\x00$\x99\xde\x8aw\xbf%\xb01\xdb&fFW\x8eh%j\xf4\xed\xb5*U\xc6\xb0\x15\xcfs\x93\x13>m\x0b\xc8\x00\x97\xe1\xe0:\xe9\xed\xea"\xc50[\xc5,\xfa=uQ\xb2\x15p\tY\xb8\x9fw\xe1\xd6\xc5\xc6yo\xe9\x12\xc0\xcd\x059\x10\x12*\xb4pm\xfe\r2\xdd\n$c\x9c\x92\xfd\xb1O

In [7]:
# Decrypt dataset
cipher = Cipher(algorithms.AES(key), modes.GCM(iv, tag), backend=default_backend())
decryptor = cipher.decryptor()
decrypted_data = decryptor.update(encrypted_data) + decryptor.finalize()

# Convert bytes back to CSV and load as DataFrame
decrypted_df = pd.read_csv(io.StringIO(decrypted_data.decode()))
print(decrypted_df)


   customer_id             name                     email  age          city  \
0            1         John Doe      john.doe@example.com   29      New York   
1            2       Jane Smith    jane.smith@example.com   34   Los Angeles   
2            3      Emily Davis   emily.davis@example.com   41       Chicago   
3            4  Michael Johnson     michael.j@example.com   50       Houston   
4            5     Sarah Wilson  sarah.wilson@example.com   25       Phoenix   
5            6   David Martinez       david.m@example.com   37  Philadelphia   
6            7   Linda Anderson       linda.a@example.com   31   San Antonio   

   annual_income  
0          75000  
1          80000  
2          67000  
3          92000  
4          62000  
5          78000  
6          68000  


In [10]:
# Save as binary file
with open("encrypted_dataset.bin", "wb") as f:
    f.write(encrypted_data)

In [1]:
import tenseal as ts

# Step 1: Create a TenSEAL context
context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree=8192, coeff_mod_bit_sizes=[60, 40, 40, 60])
context.global_scale = 2**40
context.generate_galois_keys()

# Step 2: Encrypt the dataset
data = [1.23, 4.56, 7.89]  # Example dataset
encrypted_data = ts.ckks_vector(context, data)  # Encrypt data using CKKS scheme

# Step 3: Perform computation on encrypted data
encrypted_result = encrypted_data * 2  # Multiply by 2 (done on encrypted data)

# Step 4: Decrypt the result
decrypted_result = encrypted_result.decrypt()
print("Decrypted Result:", decrypted_result)

: 

: 

In [ ]:
import pandas as pd

# Load dataset
data = pd.read_csv("data.csv")

# Convert to numeric format suitable for encryption
flattened_data = data.values.flatten().tolist()

# Encrypt using CKKS
encrypted_data = ts.ckks_vector(context, flattened_data)